In [99]:
import nannyml as nml
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.preprocessing import LabelBinarizer

In [60]:
reference_df, analysis_df, _ = nml.load_synthetic_multiclass_classification_dataset()

display(reference_df.head(3))

,acq_channel,app_behavioral_score,requested_credit_limit,app_channel,credit_bureau_score,stated_income,is_customer,timestamp,y_pred_proba_prepaid_card,y_pred_proba_highstreet_card,y_pred_proba_upmarket_card,y_pred,y_true
0,Partner3,1.808232,350,web,309,15000,True,2020-05-02 02:01:30,0.97,0.03,0.00,prepaid_card,prepaid_card
1,Partner2,4.382568,500,mobile,418,23000,True,2020-05-02 02:03:33,0.87,0.13,0.00,prepaid_card,prepaid_card
2,Partner2,-0.787575,400,web,507,24000,False,2020-05-02 02:04:49,0.47,0.35,0.18,prepaid_card,upmarket_card


In [61]:
classes = set(reference_df['y_true'].unique()).union(set(reference_df['y_pred'].unique()))

classes = list(classes)

In [62]:
cm = confusion_matrix(reference_df['y_true'], reference_df['y_pred'], labels=classes)
cm2 = confusion_matrix(reference_df['y_true'], reference_df['y_pred'], labels=classes, normalize='true')

cms = np.array([cm, cm2])

In [63]:
cms

array([[[1.51900000e+04, 2.32100000e+03, 2.42500000e+03],
        [2.13300000e+03, 1.52810000e+04, 2.68900000e+03],
        [2.59400000e+03, 2.64800000e+03, 1.47190000e+04]],

       [[7.61938202e-01, 1.16422552e-01, 1.21639246e-01],
        [1.06103567e-01, 7.60135303e-01, 1.33761130e-01],
        [1.29953409e-01, 1.32658684e-01, 7.37387906e-01]]])

In [68]:
for i in range(len(classes)):
    for j in range(len(classes)):
        print(cms[:,i,j])
        print()

[1.51900000e+04 7.61938202e-01]

[2.32100000e+03 1.16422552e-01]

[2.42500000e+03 1.21639246e-01]

[2.13300000e+03 1.06103567e-01]

[1.52810000e+04 7.60135303e-01]

[2.6890000e+03 1.3376113e-01]

[2.59400000e+03 1.29953409e-01]

[2.64800000e+03 1.32658684e-01]

[1.47190000e+04 7.37387906e-01]



In [76]:
cms[:,2,1]

array([2.64800000e+03, 1.32658684e-01])

In [3]:
estimator = nml.CBPE(
    y_pred_proba={
        'prepaid_card': 'y_pred_proba_prepaid_card',
        'highstreet_card': 'y_pred_proba_highstreet_card',
        'upmarket_card': 'y_pred_proba_upmarket_card'},
    y_pred='y_pred',
    y_true='y_true',
    timestamp_column_name='timestamp',
    problem_type='classification_multiclass',
    metrics=['roc_auc', 'f1'],
    chunk_size=6000,
)


In [4]:
estimator.fit(reference_df)



In [5]:
results = estimator.estimate(analysis_df)
display(results.filter(period='analysis').to_df())


chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:5999]           0           0      5999 2020-09-01 03:10:01   
1   [6000:11999]           1        6000     11999 2020-09-13 16:15:32   
2  [12000:17999]           2       12000     17999 2020-09-25 19:50:04   
3  [18000:23999]           3       18000     23999 2020-10-08 02:57:34   
4  [24000:29999]           4       24000     29999 2020-10-20 15:49:06   
5  [30000:35999]           5       30000     35999 2020-11-01 22:04:59   
6  [36000:41999]           6       36000     41999 2020-11-14 03:55:49   
7  [42000:47999]           7       42000     47999 2020-11-26 09:19:22   
8  [48000:53999]           8       48000     53999 2020-12-08 14:34:25   
9  [54000:59999]           9       54000     59999 2020-12-20 18:31:09   

                                  roc_auc                          ...  \
             end_date    period     value sampling_error realized  ...   
0 2020-09-13 16:15:10  analysis  0.907037       0.002143      NaN  ...   
1 2020-09-25 19:48:42  analysis  0.909948       0.002143      NaN  ...   
2 2020-10-08 02:53:47  analysis  0.909958       0.002143      NaN  ...   
3 2020-10-20 15:48:19  analysis  0.909105       0.002143      NaN  ...   
4 2020-11-01 22:04:40  analysis  0.907189       0.002143      NaN  ...   
5 2020-11-14 03:55:33  analysis  0.819515       0.002143      NaN  ...   
6 2020-11-26 09:19:06  analysis  0.820257       0.002143      NaN  ...   
7 2020-12-08 14:33:56  analysis  0.819127       0.002143      NaN  ...   
8 2020-12-20 18:30:30  analysis  0.819406       0.002143      NaN  ...   
9 2021-01-01 22:57:55  analysis  0.821584       0.002143      NaN  ...   

                                f1                          \
  lower_threshold  alert     value sampling_error realized   
0        0.900902  False  0.753301       0.005652      NaN   
1        0.900902  False  0.756422       0.005652      NaN   
2        0.900902  False  0.758166       0.005652      NaN   
3        0.900902  False  0.756557       0.005652      NaN   
4        0.900902  False  0.753618       0.005652      NaN   
5        0.900902   True  0.630985       0.005652      NaN   
6        0.900902   True  0.631482       0.005652      NaN   
7        0.900902   True  0.630552       0.005652      NaN   
8        0.900902   True  0.631736       0.005652      NaN   
9        0.900902   True  0.633443       0.005652      NaN   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  0.770258                  0.736345        0.764944   
1                  0.773378                  0.739465        0.764944   
2                  0.775122                  0.741209        0.764944   
3                  0.773514                  0.739601        0.764944   
4                  0.770575                  0.736661        0.764944   
5                  0.647941                  0.614028        0.764944   
6                  0.648439                  0.614525        0.764944   
7                  0.647509                  0.613595        0.764944   
8                  0.648692                  0.614779        0.764944   
9                  0.650400                  0.616487        0.764944   

                          
  lower_threshold  alert  
0        0.741254  False  
1        0.741254  False  
2        0.741254  False  
3        0.741254  False  
4        0.741254  False  
5        0.741254   True  
6        0.741254   True  
7        0.741254   True  
8        0.741254   True  
9        0.741254   True  

[10 rows x 23 columns]

In [6]:

metric_fig = results.plot()
metric_fig.show()

# Random

In [9]:
from sklearn.metrics import confusion_matrix

y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
print(confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"]))
print(confusion_matrix(y_true, y_pred, labels=["bird", "ant", "cat"]))
#confusion_matrix(y_true, y_pred)


[[2 0 0]
 [0 0 1]
 [1 0 2]]
[[0 0 1]
 [0 2 0]
 [0 1 2]]


In [10]:
dict1 = {'y_true':"a", 'y_pred':"a"}

for ent in dict1:
    print(ent)

y_true
y_pred


In [73]:
# make a list of the keys in the dictionary
keys = list(dict1.keys())

In [74]:
keys

['y_true', 'y_pred']

In [80]:
t = np.array([[1,2,3],[4,5,6],[7,8,9]])
t

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [82]:
d = np.sum(t, axis=1)[:,None]
d

array([[ 6],
       [15],
       [24]])

In [83]:
t/d

array([[0.16666667, 0.33333333, 0.5       ],
       [0.26666667, 0.33333333, 0.4       ],
       [0.29166667, 0.33333333, 0.375     ]])

In [84]:
np.sum(t/d, axis=1)[:,None]

array([[1.],
       [1.],
       [1.]])

In [86]:
p = np.sum(t, axis=0)[None, :]
p

array([[12, 15, 18]])

In [87]:
t/p

array([[0.08333333, 0.13333333, 0.16666667],
       [0.33333333, 0.33333333, 0.33333333],
       [0.58333333, 0.53333333, 0.5       ]])

In [88]:
np.sum(t/p, axis=0)[None, :]

array([[1., 1., 1.]])

In [91]:
# find the sum of all of t
t_sum = np.sum(t)
t_sum

45

In [96]:
a = t/t_sum
a

array([[0.02222222, 0.04444444, 0.06666667],
       [0.08888889, 0.11111111, 0.13333333],
       [0.15555556, 0.17777778, 0.2       ]])

In [97]:
a/np.sum(a)

array([[0.02222222, 0.04444444, 0.06666667],
       [0.08888889, 0.11111111, 0.13333333],
       [0.15555556, 0.17777778, 0.2       ]])

In [98]:
a

array([[0.02222222, 0.04444444, 0.06666667],
       [0.08888889, 0.11111111, 0.13333333],
       [0.15555556, 0.17777778, 0.2       ]])

In [103]:
fit_array = ['a', 'b', 'c', 'd']

lb = LabelBinarizer()
lb.fit(fit_array)

ref = lb.transform(['a', 'b', 'c'])

ana = lb.transform(['a', 'b', 'b'])

ValueError: multilabel-indicator is not supported

In [105]:
confusion_matrix = np.array([
    [10, 2, 1],
    [4, 12, 0],
    [3, 1, 15]
])

In [106]:
n_rows, n_cols = confusion_matrix.shape

In [107]:
observation_level_arrays = []

In [111]:
print(confusion_matrix[1,2])

0


In [119]:
true_proportions = np.sum(confusion_matrix, axis=1)[:,None]/np.sum(confusion_matrix)
true_proportions

array([[0.27083333],
       [0.33333333],
       [0.39583333]])

In [129]:
pred_proportions = np.sum(confusion_matrix, axis=0)[None,:]/np.sum(confusion_matrix)
pred_proportions

array([[0.35416667, 0.3125    , 0.33333333]])

In [143]:
pred_marginal = np.sum(confusion_matrix, axis=0)[None,:]
true_marginal = np.sum(confusion_matrix, axis=1)[:,None]

print(pred_marginal)
print(true_marginal)
print()

# get the 2nd value in the pred_marginal array
print(pred_marginal[0,1])
# get the 2nd value in the true_marginal array
print(true_marginal[1,0])

[[17 15 16]]
[[13]
 [16]
 [19]]

15
16


In [130]:
#intitalize an array called stds witht the same dimensions as the confusion matrix

stds = np.zeros((n_rows, n_cols))



for i in range(n_rows):
    for j in range(n_cols):
        # Create an observation level array for each cell
        obs_level_array = np.zeros(confusion_matrix.sum(), dtype=int)
        end_index = confusion_matrix[i, j]
        obs_level_array[0:end_index] = 1

        std = np.std(obs_level_array)

        stds[i,j] = std


stds

array([[0.40611643, 0.19982631, 0.14282614],
       [0.2763854 , 0.4330127 , 0.        ],
       [0.24206146, 0.14282614, 0.46351241]])

In [122]:
pred_normed_SE = stds/np.sqrt(np.sum(confusion_matrix)*pred_proportions)
pred_normed_SE

array([[0.05861786, 0.02884244, 0.02061518],
       [0.0398928 , 0.0625    , 0.        ],
       [0.03493856, 0.02061518, 0.06690225]])

In [125]:
pp = np.array([[0.35416667, 0.3125    , 0.33333333]])
10 * pp

array([[3.5416667, 3.125    , 3.3333333]])

In [126]:
we = np.array([[10, 20    , 30]])

In [127]:
wx = np.array([
    [10, 10, 10],
    [10, 10, 10],
    [10, 10, 10]
])

In [133]:
wx/(we*10)

array([[0.1       , 0.05      , 0.03333333],
       [0.1       , 0.05      , 0.03333333],
       [0.1       , 0.05      , 0.03333333]])

In [146]:
x = 'alsl'

if x in [None, "all"]:
    print("yes")

In [147]:
np.ones(3)

array([1., 1., 1.])

In [148]:
confusion_matrix

array([[10,  2,  1],
       [ 4, 12,  0],
       [ 3,  1, 15]])

In [149]:
confusion_matrix*100

array([[1000,  200,  100],
       [ 400, 1200,    0],
       [ 300,  100, 1500]])